In [1]:
import pandas as pd
import json
from google.colab import files

# Subir el archivo JSON
uploaded = files.upload()

# Obtener el nombre del archivo
filename = list(uploaded.keys())[0]

# Leer el JSON
df = pd.read_json(filename)

# Mostrar las primeras filas
print("Datos cargados:")
print(df.head())

# Guardar como CSV
csv_filename = filename.replace('.json', '.csv')
df.to_csv(csv_filename, index=False)

print(f"Archivo guardado como: {csv_filename}")

# Descargar el CSV
files.download(csv_filename)

Saving TelecomX_Data.json to TelecomX_Data (1).json
Datos cargados:
   customerID Churn                                           customer  \
0  0002-ORFBO    No  {'gender': 'Female', 'SeniorCitizen': 0, 'Part...   
1  0003-MKNFE    No  {'gender': 'Male', 'SeniorCitizen': 0, 'Partne...   
2  0004-TLHLJ   Yes  {'gender': 'Male', 'SeniorCitizen': 0, 'Partne...   
3  0011-IGKFF   Yes  {'gender': 'Male', 'SeniorCitizen': 1, 'Partne...   
4  0013-EXCHZ   Yes  {'gender': 'Female', 'SeniorCitizen': 1, 'Part...   

                                             phone  \
0   {'PhoneService': 'Yes', 'MultipleLines': 'No'}   
1  {'PhoneService': 'Yes', 'MultipleLines': 'Yes'}   
2   {'PhoneService': 'Yes', 'MultipleLines': 'No'}   
3   {'PhoneService': 'Yes', 'MultipleLines': 'No'}   
4   {'PhoneService': 'Yes', 'MultipleLines': 'No'}   

                                            internet  \
0  {'InternetService': 'DSL', 'OnlineSecurity': '...   
1  {'InternetService': 'DSL', 'OnlineSecurity': '.

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [8]:
# Preprocesamiento adicional para modelado
def preprocesamiento_avanzado(df_limpio, target_col=None):
    """
    Preprocesamiento avanzado para modelado predictivo
    """
    print("\n" + "="*50)
    print("⚙️  PREPROCESAMIENTO AVANZADO")
    print("="*50)

    # 1. Manejo de valores nulos
    print("1. Manejo de valores nulos:")
    for col in df_limpio.columns:
        if df_limpio[col].isnull().sum() > 0:
            if df_limpio[col].dtype in ['object', 'category']:
                # Para categóricas: moda
                moda = df_limpio[col].mode()[0]
                df_limpio[col].fillna(moda, inplace=True)
                print(f"   {col}: nulos llenados con moda ('{moda}')")
            else:
                # Para numéricas: mediana
                mediana = df_limpio[col].median()
                df_limpio[col].fillna(mediana, inplace=True)
                print(f"   {col}: nulos llenados con mediana ({mediana:.2f})")

    # 2. Codificación de variables categóricas
    print("\n2. Variables categóricas:")
    categorical_cols = df_limpio.select_dtypes(include=['object']).columns
    if len(categorical_cols) > 0:
        print(f"   {len(categorical_cols)} variables categóricas encontradas")
        for col in categorical_cols:
            print(f"   {col}: {df_limpio[col].nunique()} categorías")
    else:
        print("   No hay variables categóricas")

    return df_limpio

# Ejecutar preprocesamiento avanzado
df_preprocesado = preprocesamiento_avanzado(df_limpio.copy(), target_col)

# Guardar dataset preprocesado
nombre_preprocesado = filename.replace('.json', '_preprocesado.csv')
df_preprocesado.to_csv(nombre_preprocesado, index=False)
files.download(nombre_preprocesado)

print(f"\n🎉 Proceso completado! Dataset listo para modelado predictivo.")

NameError: name 'df_limpio' is not defined

In [7]:
import pandas as pd
import numpy as np
import json
from google.colab import files
import matplotlib.pyplot as plt
import seaborn as sns

# Configuración
pd.set_option('display.max_columns', None)
plt.style.use('default')
sns.set_palette("husl")

# 1. Subir el archivo JSON
print("📤 Por favor, sube tu archivo JSON")
uploaded = files.upload()
filename = list(uploaded.keys())[0]

# 2. Leer el archivo JSON
def cargar_json(filename):
    try:
        # Intentar leer directamente con pandas
        try:
            df = pd.read_json(filename)
            print("✅ JSON leído directamente con pd.read_json()")
            return df
        except:
            # Leer manualmente si falla
            with open(filename, 'r', encoding='utf-8') as f:
                data = json.load(f)

            # Convertir según la estructura
            if isinstance(data, list):
                df = pd.json_normalize(data)  # Para JSON anidados
            elif isinstance(data, dict):
                df = pd.DataFrame([data])
            else:
                raise ValueError("Formato JSON no soportado")

            print("✅ JSON leído con json.load() y convertido")
            return df

    except Exception as e:
        print(f"❌ Error al leer el JSON: {e}")
        return None

df = cargar_json(filename)

if df is None:
    # Intentar con diferentes enfoques
    print("⚠️  Intentando método alternativo...")
    try:
        df = pd.read_json(filename, lines=True)
        print("✅ JSON leído con lines=True")
    except:
        with open(filename, 'r') as f:
            data = [json.loads(line) for line in f]
        df = pd.json_normalize(data)
        print("✅ JSON leído línea por línea")

print(f"\n📊 Dataset cargado: {df.shape}")

# 3. Mostrar información inicial
print(f"\n📋 COLUMNAS ORIGINALES ({len(df.columns)}):")
for i, col in enumerate(df.columns, 1):
    print(f"   {i:2d}. {col}")

print(f"\n🔍 TIPOS DE DATOS:")
print(df.dtypes.value_counts())

# 4. Función de análisis detallado para eliminación de columnas
def analizar_y_eliminar_columnas(df):
    """
    Analiza y elimina columnas que no aportan valor predictivo
    """
    print("\n" + "="*60)
    print("🔍 ANÁLISIS PARA ELIMINACIÓN DE COLUMNAS")
    print("="*60)

    # Patrones de columnas a eliminar (específicos para cancelación/churn)
    patrones_eliminar = [
        # Identificadores únicos
        'id', '_id', 'id_', 'uuid', 'guid', 'hash', 'token', 'key',
        'customer', 'user', 'account', 'client', 'subscriber',
        'number', 'num', 'no', 'code', 'ref', 'reference',

        # Información personal
        'name', 'first', 'last', 'fullname', 'email', 'phone',
        'mobile', 'tel', 'address', 'zip', 'postal', 'location',

        # Fechas y timestamps (a menos que se transformen)
        'date', 'time', 'timestamp', 'created', 'updated', 'modified',
        'start', 'end', 'expir', 'register',

        # Campos técnicos/metadatos
        'version', 'status', 'type', 'category', 'level',
        'flag', 'indicator', 'marker',

        # Texto libre no estructurado
        'comment', 'note', 'description', 'detail', 'remark',
        'feedback', 'review', 'message', 'text',

        # Campos específicos de sistemas
        'session', 'device', 'ip', 'mac', 'imei', 'imsi',
        'browser', 'os', 'platform'
    ]

    columnas_a_eliminar = []
    razones_eliminacion = {}

    print("\n📝 ANALIZANDO CADA COLUMNA:")

    for columna in df.columns:
        col_lower = columna.lower()
        unique_ratio = df[columna].nunique() / len(df)
        null_ratio = df[columna].isnull().mean()

        # Verificar múltiples criterios para eliminar
        razones = []

        # 1. Patrones en el nombre
        if any(patron in col_lower for patron in patrones_eliminar):
            razones.append("Patrón no predictivo")

        # 2. Valores únicos (demasiados o muy pocos)
        if unique_ratio == 1.0:
            razones.append("Todos los valores iguales")
        elif unique_ratio > 0.95 and df[columna].dtype == 'object':
            razones.append("Demasiados valores únicos")
        elif unique_ratio < 0.05:
            razones.append("Muy pocos valores únicos")

        # 3. Valores nulos
        if null_ratio > 0.7:
            razones.append("Demasiados valores nulos")

        # 4. Si es identificador único
        if ('id' in col_lower or 'code' in col_lower) and unique_ratio > 0.8:
            razones.append("Posible identificador único")

        if razones:
            columnas_a_eliminar.append(columna)
            razones_eliminacion[columna] = razones
            print(f"   ❌ {columna:25} -> ELIMINAR ({', '.join(razones)})")
        else:
            print(f"   ✅ {columna:25} -> MANTENER")

    return columnas_a_eliminar, razones_eliminacion

# 5. Ejecutar análisis y eliminación
columnas_eliminar, razones = analizar_y_eliminar_columnas(df)

# 6. Eliminar columnas
df_limpio = df.drop(columns=columnas_eliminar, errors='ignore')

print(f"\n📈 RESULTADOS DE LA LIMPIEZA:")
print(f"   • Columnas originales: {df.shape[1]}")
print(f"   • Columnas eliminadas: {len(columnas_eliminar)}")
print(f"   • Columnas finales: {df_limpio.shape[1]}")
print(f"   • Reducción: {(len(columnas_eliminar)/df.shape[1])*100:.1f}%")

# 7. Mostrar columnas mantenidas
print(f"\n🎯 COLUMNAS MANTENIDAS ({df_limpio.shape[1]}):")
for i, col in enumerate(df_limpio.columns, 1):
    print(f"   {i:2d}. {col}")

# 8. Análisis específico para predicción de cancelación
def analisis_cancelacion(df_limpio):
    """
    Análisis específico para modelos de predicción de cancelación
    """
    print("\n" + "="*50)
    print("🔥 ANÁLISIS PARA PREDICCIÓN DE CANCELACIÓN")
    print("="*50)

    # Buscar variable target
    target_candidates = ['churn', 'cancel', 'attrition', 'abandon', 'churned', 'cancelled']
    target_col = None

    for candidate in target_candidates:
        for col in df_limpio.columns:
            if candidate in col.lower():
                target_col = col
                break
        if target_col:
            break

    if target_col:
        print(f"✅ Variable target identificada: '{target_col}'")

        # Distribución de la variable target
        target_dist = df_limpio[target_col].value_counts(normalize=True)
        print(f"\n📊 Distribución de {target_col}:")
        for valor, porcentaje in target_dist.items():
            print(f"   {valor}: {porcentaje:.2%}")

        # Visualización
        plt.figure(figsize=(12, 5))

        plt.subplot(1, 2, 1)
        target_dist.plot(kind='bar', color=['lightgreen', 'lightcoral'])
        plt.title(f'Distribución de {target_col}')
        plt.xticks(rotation=0)

        plt.subplot(1, 2, 2)
        plt.pie(target_dist.values, labels=target_dist.index,
                autopct='%1.1f%%', colors=['lightblue', 'lightpink'])
        plt.title(f'Proporción de {target_col}')

        plt.tight_layout()
        plt.show()

        return target_col
    else:
        print("⚠️  No se encontró variable target clara de cancelación")
        print("   Buscando columnas potenciales...")

        # Buscar columnas binarias que podrían ser el target
        binary_cols = []
        for col in df_limpio.columns:
            if df_limpio[col].nunique() == 2:
                binary_cols.append(col)

        if binary_cols:
            print(f"   Columnas binarias encontradas: {binary_cols}")
        else:
            print("   No se encontraron columnas binarias")

        return None

# 9. Ejecutar análisis de cancelación
target_col = analisis_cancelacion(df_limpio)

# 10. Guardar el dataset limpio
nombre_limpio = filename.replace('.json', '_limpio.csv')
df_limpio.to_csv(nombre_limpio, index=False, encoding='utf-8')

print(f"\n💾 Dataset limpio guardado como: {nombre_limpio}")

# 11. Descargar el archivo
files.download(nombre_limpio)
print("📥 Archivo descargado a tu computadora")

# 12. Información final del dataset limpio
print(f"\n📋 INFORMACIÓN FINAL DEL DATASET LIMPIO:")
print(f"Forma: {df_limpio.shape}")
print(f"\nTipos de datos:")
print(df_limpio.dtypes.value_counts())

print(f"\n📊 Primeras 3 filas:")
print(df_limpio.head(3))

# 13. Análisis de valores nulos en el dataset limpio
print(f"\n🔍 VALORES NULOS EN DATASET LIMPIO:")
nulos = df_limpio.isnull().sum()
nulos_pct = (nulos / len(df_limpio)) * 100

if nulos.sum() > 0:
    print("Columnas con valores nulos:")
    for col in nulos_pct[nulos_pct > 0].sort_values(ascending=False).index:
        print(f"   {col:20}: {nulos_pct[col]:.1f}% nulos")
else:
    print("✅ No hay valores nulos")

📤 Por favor, sube tu archivo JSON


Saving TelecomX_Data.json to TelecomX_Data (3).json
✅ JSON leído directamente con pd.read_json()

📊 Dataset cargado: (7267, 6)

📋 COLUMNAS ORIGINALES (6):
    1. customerID
    2. Churn
    3. customer
    4. phone
    5. internet
    6. account

🔍 TIPOS DE DATOS:
object    6
Name: count, dtype: int64

🔍 ANÁLISIS PARA ELIMINACIÓN DE COLUMNAS

📝 ANALIZANDO CADA COLUMNA:
   ❌ customerID                -> ELIMINAR (Patrón no predictivo, Todos los valores iguales, Posible identificador único)
   ❌ Churn                     -> ELIMINAR (Muy pocos valores únicos)


TypeError: unhashable type: 'dict'

In [2]:
import pandas as pd
import json
from google.colab import files

# Subir el archivo
uploaded = files.upload()
filename = list(uploaded.keys())[0]

# Leer el JSON
with open(filename, 'r') as f:
    data = json.load(f)

# Si es una lista de diccionarios
if isinstance(data, list):
    df = pd.json_normalize(data)  # Para JSON anidados
else:
    df = pd.DataFrame([data])

# Mostrar información
print(f"Forma del DataFrame: {df.shape}")
print("\nPrimeras filas:")
print(df.head())

# Guardar como CSV
csv_filename = filename.replace('.json', '.csv')
df.to_csv(csv_filename, index=False, encoding='utf-8')

print(f"Archivo CSV guardado: {csv_filename}")

# Opcional: descargar
files.download(csv_filename)

Saving TelecomX_Data.json to TelecomX_Data (2).json
Forma del DataFrame: (7267, 21)

Primeras filas:
   customerID Churn customer.gender  customer.SeniorCitizen customer.Partner  \
0  0002-ORFBO    No          Female                       0              Yes   
1  0003-MKNFE    No            Male                       0               No   
2  0004-TLHLJ   Yes            Male                       0               No   
3  0011-IGKFF   Yes            Male                       1              Yes   
4  0013-EXCHZ   Yes          Female                       1              Yes   

  customer.Dependents  customer.tenure phone.PhoneService phone.MultipleLines  \
0                 Yes                9                Yes                  No   
1                  No                9                Yes                 Yes   
2                  No                4                Yes                  No   
3                  No               13                Yes                  No   
4            

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [5]:

# Leer el archivodf = pd.read_csv(TelecomX_Data(2))
print("Forma original del dataset:", df.shape)
print("\nColumnas originales:")
print(df.columns.tolist())

# Columnas que typically no aportan valor (ajusta según tu caso)
columnas_a_eliminar = [
    'id', 'customer_id', 'client_id', 'user_id', 'account_id',
    'transaction_id', 'order_id', 'invoice_id', 'record_id',
    'uuid', 'guid', 'hash', 'token', 'session_id',
    'created_at', 'updated_at', 'deleted_at',  # Timestamps si no son relevantes
    'url', 'link', 'reference', 'code',
    'name', 'first_name', 'last_name',  # Identificadores personales
    'email', 'phone', 'address',  # Información personal
    'notes', 'comments', 'description'  # Texto libre no estructurado
]

# Filtrar solo las columnas que existen en el dataset
columnas_existentes = [col for col in columnas_a_eliminar if col in df.columns]

# Eliminar columnas
df_limpio = df.drop(columns=columnas_existentes, errors='ignore')

print(f"\nColumnas eliminadas: {columnas_existentes}")
print("Forma después de eliminar columnas:", df_limpio.shape)
print("\nColumnas restantes:")
print(df_limpio.columns.tolist())

NameError: name 'TelecomX_Data' is not defined

In [6]:
import pandas as pd
import numpy as np
from google.colab import files
import matplotlib.pyplot as plt
import seaborn as sns

# Configuración de visualización
plt.style.use('default')
sns.set_palette("husl")

# 1. Subir el archivo CSV específico
print("📤 Por favor, sube el archivo 'TelecomX_Data(2).csv'")
uploaded = files.upload()
filename = 'TelecomX_Data(2).csv'  # Nombre específico del archivo

# 2. Leer el archivo CSV
try:
    df = pd.read_csv(filename, encoding='utf-8')
    print("✅ Archivo TelecomX_Data(2).csv cargado exitosamente")
except UnicodeDecodeError:
    # Intentar con diferentes encodings si hay problemas
    try:
        df = pd.read_csv(filename, encoding='latin-1')
        print("✅ Archivo cargado con encoding latin-1")
    except:
        df = pd.read_csv(filename, encoding='ISO-8859-1')
        print("✅ Archivo cargado con encoding ISO-8859-1")

# 3. Información inicial del dataset
print(f"\n📊 INFORMACIÓN DEL DATASET TELECOMX")
print("="*50)
print(f"📦 Filas: {df.shape[0]}")
print(f"🏷️  Columnas: {df.shape[1]}")
print(f"\n📋 Lista de columnas:")
for i, col in enumerate(df.columns, 1):
    print(f"   {i:2d}. {col}")

# 4. Análisis específico para datos de telecomunicaciones
def analisis_telecom(df):
    print("\n" + "="*50)
    print("🔍 ANÁLISIS ESPECÍFICO PARA TELECOMUNICACIONES")
    print("="*50)

    # Patrones comunes en telecom que suelen eliminarse
    patrones_telecom_eliminar = [
        # Identificadores
        'customerid', 'userid', 'accountid', 'subscriberid', 'clientid',
        'id', '_id', 'id_', 'uuid', 'guid',

        # Información personal
        'name', 'nombre', 'firstname', 'lastname', 'fullname',
        'email', 'phone', 'telefono', 'mobile', 'celular',
        'address', 'direccion', 'zip', 'postalcode',

        # Fechas específicas (a menos que se transformen)
        'registrationdate', 'signupdate', 'joindate',

        # Campos técnicos no predictivos
        'sessionid', 'deviceid', 'imei', 'imsi',

        # Texto libre
        'comments', 'notes', 'description', 'feedback'
    ]

    columnas_a_eliminar = []
    columnas_a_mantener = []

    print("\n📝 Análisis por columna:")
    for columna in df.columns:
        col_lower = columna.lower()

        # Estadísticas
        unique_ratio = df[columna].nunique() / len(df)
        null_ratio = df[columna].isnull().mean()

        # Verificar patrones de eliminación
        eliminar = False
        razon = ""

        if any(patron in col_lower for patron in patrones_telecom_eliminar):
            eliminar = True
            razon = "Patrón de telecom para eliminar"
        elif unique_ratio == 1.0:
            eliminar = True
            razon = "Todos los valores iguales"
        elif null_ratio > 0.8:
            eliminar = True
            razon = f"Demasiados nulos ({null_ratio:.1%})"
        elif unique_ratio > 0.95 and df[columna].dtype == 'object':
            eliminar = True
            razon = f"Demasiados únicos ({unique_ratio:.1%})"

        if eliminar:
            columnas_a_eliminar.append(columna)
            print(f"   ❌ {columna:25} -> ELIMINAR ({razon})")
        else:
            columnas_a_mantener.append(columna)
            print(f"   ✅ {columna:25} -> MANTENER")

    return columnas_a_eliminar, columnas_a_mantener

# 5. Ejecutar análisis específico
columnas_eliminar, columnas_mantener = analisis_telecom(df)

# 6. Eliminar columnas identificadas
df_limpio = df.drop(columns=columnas_eliminar, errors='ignore')

print(f"\n📈 RESULTADOS DE LA LIMPIEZA:")
print(f"   • Columnas originales: {df.shape[1]}")
print(f"   • Columnas eliminadas: {len(columnas_eliminar)}")
print(f"   • Columnas finales: {df_limpio.shape[1]}")
print(f"   • Reducción: {(len(columnas_eliminar)/df.shape[1])*100:.1f}%")

# 7. Información del dataset limpio
print(f"\n🎯 COLUMNAS MANTENIDAS ({len(columnas_mantener)}):")
for i, col in enumerate(columnas_mantener, 1):
    print(f"   {i:2d}. {col}")

# 8. Análisis de variables importantes para churn (si existe)
if any(col in df_limpio.columns for col in ['churn', 'Churn', 'cancelacion', 'attrition']):
    target_col = next((col for col in ['churn', 'Churn', 'cancelacion', 'attrition']
                      if col in df_limpio.columns), None)

    print(f"\n🔥 ANÁLISIS DE CHURN DETECTADO")
    print(f"   Variable target: {target_col}")

    # Distribución del churn
    churn_dist = df_limpio[target_col].value_counts(normalize=True)
    print(f"\n📊 Distribución de {target_col}:")
    for valor, porcentaje in churn_dist.items():
        print(f"   {valor}: {porcentaje:.2%}")

    # Visualización rápida
    plt.figure(figsize=(10, 6))

    plt.subplot(1, 2, 1)
    churn_dist.plot(kind='bar', color=['skyblue', 'salmon'])
    plt.title(f'Distribución de {target_col}')
    plt.xticks(rotation=0)

    plt.subplot(1, 2, 2)
    plt.pie(churn_dist.values, labels=churn_dist.index, autopct='%1.1f%%', colors=['lightblue', 'lightcoral'])
    plt.title(f'Proporción de {target_col}')

    plt.tight_layout()
    plt.show()

# 9. Guardar el dataset limpio
nombre_limpio = 'TelecomX_Data_Limpio.csv'
df_limpio.to_csv(nombre_limpio, index=False, encoding='utf-8')

print(f"\n💾 Dataset limpio guardado como: {nombre_limpio}")

# 10. Descargar el archivo
files.download(nombre_limpio)
print("📥 Archivo descargado a tu computadora")

# 11. Información adicional
print(f"\n📋 INFORMACIÓN DEL DATASET LIMPIO:")
print(df_limpio.info())
print(f"\n📊 Primeras 3 filas:")
print(df_limpio.head(3))

print(f"\n🔍 Tipos de datos finales:")
print(df_limpio.dtypes.value_counts())

# 12. Estadísticas básicas de columnas numéricas
if df_limpio.select_dtypes(include=[np.number]).shape[1] > 0:
    print(f"\n📈 Estadísticas de columnas numéricas:")
    print(df_limpio.select_dtypes(include=[np.number]).describe())

📤 Por favor, sube el archivo 'TelecomX_Data(2).csv'


Saving TelecomX_Data (2).csv to TelecomX_Data (2) (1).csv


FileNotFoundError: [Errno 2] No such file or directory: 'TelecomX_Data(2).csv'